<a href="https://colab.research.google.com/github/park-geun-hyeong/TimeSeries/blob/main/Preprocessing/CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
%matplotlib inline
sns.set_style('whitegrid')

import os
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
import sklearn
from sklearn.linear_model import ElasticNet
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
cmap_data = plt.cm.Paired
cmap_cv= plt.cm.coolwarm

n_splits = 5

path = '/content/drive/MyDrive/TimeSeries/2week/'
os.chdir(path)

In [ ]:
df = pd.read_csv('Gemini_ETHUSD_d.csv')
df = df.drop(['Symbol', 'Unix Timestamp'], axis=1).set_index('Date').sort_index()
df

,Open,High,Low,Close,Volume ETH,Volume USD
Date,,,,,,
2016-05-09,12.00,12.00,9.36,9.98,1317.900,12885.06
2016-05-10,9.98,9.98,9.36,9.68,672.060,6578.20
2016-05-11,9.68,10.47,9.68,10.43,3052.510,30978.11
2016-05-12,10.43,12.00,9.92,10.20,2072.560,22183.39
2016-05-13,10.20,11.59,10.20,10.69,1769.710,18923.55
...,...,...,...,...,...,...
2020-11-16,448.97,466.22,446.16,460.73,10214.630,4680880.22
2020-11-17,460.73,485.05,460.23,482.81,21579.440,10262735.24
2020-11-18,482.81,495.76,459.14,479.49,32772.610,15638074.52


In [ ]:
for i in range(1, 10):
    col_name = f"{i}d_FWD_close"
    df[col_name] = df.Close.shift(-i)

df = df.dropna()
df

,Open,High,Low,Close,Volume ETH,Volume USD,1d_FWD_close,2d_FWD_close,3d_FWD_close,4d_FWD_close,5d_FWD_close,6d_FWD_close,7d_FWD_close,8d_FWD_close,9d_FWD_close
Date,,,,,,,,,,,,,,,
2016-05-09,12.00,12.00,9.36,9.98,1317.90,12885.06,9.68,10.43,10.20,10.69,10.25,10.06,11.37,12.23,13.30
2016-05-10,9.98,9.98,9.36,9.68,672.06,6578.20,10.43,10.20,10.69,10.25,10.06,11.37,12.23,13.30,14.50
2016-05-11,9.68,10.47,9.68,10.43,3052.51,30978.11,10.20,10.69,10.25,10.06,11.37,12.23,13.30,14.50,13.90
2016-05-12,10.43,12.00,9.92,10.20,2072.56,22183.39,10.69,10.25,10.06,11.37,12.23,13.30,14.50,13.90,13.97
2016-05-13,10.20,11.59,10.20,10.69,1769.71,18923.55,10.25,10.06,11.37,12.23,13.30,14.50,13.90,13.97,14.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-07,456.46,468.76,424.59,435.60,19540.00,8731201.76,454.57,444.51,451.00,463.81,463.94,476.92,461.19,448.97,460.73
2020-11-08,435.60,460.03,432.89,454.57,8005.81,3583875.50,444.51,451.00,463.81,463.94,476.92,461.19,448.97,460.73,482.81
2020-11-09,454.57,459.01,434.24,444.51,13506.84,6046992.67,451.00,463.81,463.94,476.92,461.19,448.97,460.73,482.81,479.49


In [ ]:
x = df.iloc[:, :6]
y = df.iloc[:, 6:]

split = int(len(df) * 0.7)

x_train = x[:split]
x_test = x[split:]

y_train = y[:split]
y_test = y[split:]

In [ ]:
def build_model(_alpha, _l1_ratio):
    estimator = ElasticNet(
        alpha = _alpha,
        l1_ratio = _l1_ratio,
        fit_intercept=True,
        normalize =False,
        precompute = False,
        max_iter = 16,
        copy_X=True,
        tol=0.1,
        warm_start =False,
        positive = False,
        random_state=None,
        selection = 'random'
    )

    return MultiOutputRegressor(estimator, n_jobs = 4)

## Roling

In [ ]:
model = build_model(1.0, 0.3)
tscv = TimeSeriesSplit(n_splits = 5) ## Roling based
rmse = np.sqrt(-cross_val_score(model, x_train, y_train, cv =tscv, scoring = 'neg_mean_squared_error'))
R2 = cross_val_score(model, x_train, y_train, cv =tscv ,scoring = 'r2')

print(f"RMSE : {rmse.mean():.5f} (+/- {rmse.std():.5f})")
print(f"R2 : {R2.mean():.5f} (+/- {R2.std():.5f})")

RMSE : 68.19833 (+/- 44.63060)
R2 : 0.40228 (+/- 0.59800)


## Blocking

In [ ]:
class BlockingTimeSeriesSplit():

    def __init__(self, n_splits):
        self.n_splits = n_splits

    def get_n_splits(self, x, y, gropus):
        return self.n_splits

    def split(self, x, y = None, gropus =None):
        n_sample = len(x)
        k_fold_size = n_sample // self.n_splits
        indices = np.arange(n_sample)

        margin = 0
        for i in range(self.n_splits):
            start = i*k_fold_size
            stop = start + k_fold_size
            mid = int(0.5 * (stop - start)) + start

            yield indices[start:mid], indices[mid+ margin : stop]


In [ ]:
btscv = BlockingTimeSeriesSplit(5)
rmse = np.sqrt(-cross_val_score(model, x_train, y_train, cv = btscv, scoring ='neg_mean_squared_error'))
R2 = cross_val_score(model, x_train, y_train, cv = btscv, scoring = 'r2')

print(f"RMSE : {rmse.mean():.5f} (+/- {rmse.std():.5f})")
print(f"R2 : {R2.mean():.5f} (+/- {R2.std():.5f})")

RMSE : 69.15579 (+/- 50.30010)
R2 : 0.10550 (+/- 0.48741)
